# Homework 6: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [48]:
library(tidyverse)
library(here)
library(skimr)
library(janitor)
library(psych)
library(reshape2)
library(dplyr)

setwd("~/Documents/GitHub/Luetkehans_DSPN_S22/data")

lexdata <- read.csv("LexicalData.csv")
items <- read.csv("Items.csv")

head(lexical_data)
head(items)


,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,"1,094",refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<fct>,<int>,<fct>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


In [49]:
lexdata <- lexdata %>% 
mutate(D_RT_clean = as.numeric(gsub(",", "", as.character(D_RT)))) %>%
na.omit() %>%
subset(select = -c(D_RT))

head(lexdata)

colnames(lexdata) <- c('Sub_ID', 'Trial', 'Type', 'Word', 'Outlier', 'D_Zscore', 'D_RT')

lexdata <- left_join(lexdata, items, by = "Word") #%>%
#subset(select = -c(Occurrences, Freq_HAL))

#For some reason, trying to remove the columns that we don't need breaks the code?  
#Does it matter if I still have occurrences and Freq_HAL?  

head(lexdata)


,Sub_ID,Trial,Type,D_Word,Outlier,D_Zscore,D_RT_clean
,<int>,<int>,<int>,<fct>,<fct>,<dbl>,<dbl>
1,157,1,1,browse,false,-0.437,710
2,67,1,1,refrigerant,false,0.825,1094
3,120,1,1,gaining,false,-0.645,587
4,21,1,1,cheerless,false,0.025,984
5,236,1,1,pattered,false,-0.763,577
6,236,2,1,conjures,false,-0.364,715


,Sub_ID,Trial,Type,Word,Outlier,D_Zscore,D_RT,Occurrences,Length,Freq_HAL,Log_Freq_HAL
,<int>,<int>,<int>,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>,<fct>,<dbl>
1,157,1,1,browse,false,-0.437,710,2,6,"7,016",8.856
2,67,1,1,refrigerant,false,0.825,1094,3,11,104,4.644
3,120,1,1,gaining,false,-0.645,587,4,7,"4,039",8.304
4,21,1,1,cheerless,false,0.025,984,4,9,14,2.639
5,236,1,1,pattered,false,-0.763,577,4,8,4,1.386
6,236,2,1,conjures,false,-0.364,715,4,8,194,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [51]:
lexlinearmodel <- lm(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, data = lexdata)

summary(lexlinearmodel)




Call:
lm(formula = D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, 
    data = lexdata)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Call:
lm(formula = D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, 
    data = lexdata)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***

Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 

Length               47.7531     1.6368  29.175  < 2e-16 ***

Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***

---
Signif. codes:  
0 ‘***’ 
0.001 ‘**’ 
0.01 ‘*’ 
0.05 ‘.’ 
0.1 ‘ ’ 

Residual standard error: 

359.1 on 62606 degrees of freedom

Multiple R-squared:  0.09473,	
Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  
p-value: < 2.2e-16

Now, install `lme4` using `install.packages()` and then load the library. 

In [1]:
install.packages('lme4')
library(lme4)

#Why won't this install???


also installing the dependency ‘nloptr’





  There is a binary version available but the source version is later:
        binary source needs_compilation
nloptr 1.2.2.3  2.0.0              TRUE


The downloaded binary packages are in
	/var/folders/9x/yng26nj50sq1bzz3bjr6bgkm0000gn/T//Rtmp38nN5g/downloaded_packages


installing the source package ‘nloptr’


Warning message in install.packages("lme4"):
“installation of package ‘nloptr’ had non-zero exit status”
Warning message:
“package ‘lme4’ was built under R version 3.6.2”
Loading required package: Matrix



ERROR: Error: package or namespace load failed for ‘lme4’ in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called ‘nloptr’


Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [57]:
lexlinearmodel_mixed <- lmer(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1|Sub_ID), data = lexdata)

summary(lexlinearmodel_mixed)



ERROR: Error in lmer(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1 | : could not find function "lmer"


---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> *Write your response here* 
> 

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [ ]:
# WRITE YOUR CODE HERE



> *Write your response here* 
> 

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> *Write your response here* 
> 

**DUE:** 5pm EST, March 25, 2022

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*